# Batch — Tropical Tree Cover (4 tiles) → Máscara ≥ 40% (10 m)
Este notebook baixa e processa **apenas** os tiles:
`10S_050W, 10S_060W, 20S_050W, 20S_060W`

Pipeline resumido:
1. Ler CSV/GeoJSON de subset (ou o GeoJSON original, se preferir);
2. Baixar os **GeoTIFF F10m** de cada tile (com opção de **Bearer Token**);
3. Aplicar **threshold de 40%** em **chunks** (sem estourar memória);
4. Salvar **mask.tif** (uint8, tiled, LZW) e **mask.png** por tile;
5. (Opcional) Downsample via `overview` para arquivos menores.

> Observação: este ambiente de demonstração não tem internet. Execute este notebook na sua máquina (ou Colab) para baixar os rasters.


## Requisitos
```bash
#pip install rasterio shapely pyproj requests numpy matplotlib

pip install -r requirements.txt
```


In [4]:
# ## Configurações
import os, json
from pathlib import Path

# Arquivos de entrada (CSV e GeoJSON) foram baixados do site de forma manual
# https://lpdaacsvc.cr.usgs.gov/appeears/products/collection/10
# https://data.globalforestwatch.org/datasets/gfw::tropical-tree-cover/explore
TILES_CSV = r"outputs/Tropical_Tree_Cover.csv"
SUBSET_GEOJSON = r"outputs/Tropical_Tree_Cover.geojson"

# Diretórios de saída
OUT_DIR = Path("./outputs/tropical_tree_cover_data").resolve()
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Lista explícita de tiles
TILES_WANTED = ["10S_050W", "10S_060W", "20S_050W", "20S_060W"]

# Parâmetros de processamento
TIMEOUT = 300 # segundos para requests

# Autenticação opcional (se tomar 403 nas URLs)
USE_BEARER = False
BEARER_TOKEN = "COLOQUE_SEU_TOKEN_AQUI"  # só se USE_BEARER=True

print("Saídas em:", OUT_DIR)


Saídas em: C:\git\NASA\DataRoots\DataRoots\outputs\tropical_tree_cover_data


In [6]:
# %%
import pandas as pd
import requests

def make_headers():
    if USE_BEARER and BEARER_TOKEN and BEARER_TOKEN.strip():
        return {"Authorization": f"Bearer {BEARER_TOKEN.strip()}", "User-Agent": "DataRoots/TileProcessor"}
    return {"User-Agent": "DataRoots/TileProcessor"}

def download_file(url, dest_path, chunk=1024*1024):
    dest_path = str(dest_path)
    headers = make_headers()
    with requests.get(url, headers=headers, stream=True, timeout=TIMEOUT) as r:
        r.raise_for_status()
        with open(dest_path, "wb") as f:
            for part in r.iter_content(chunk_size=chunk):
                if part:
                    f.write(part)
    return dest_path

def process_tile(tile_id, url):
    # 1) Download
    tif_local = OUT_DIR / f"{tile_id}_F10m.tif"
    if not tif_local.exists():
        print(f"[{tile_id}] Baixando...")
        try:
            download_file(url, tif_local)
        except Exception as e:
            print(f"[{tile_id}] Falha no download:", e)
            return None
    else:
        print(f"[{tile_id}] Já existe:", tif_local.name)


    return {"tile_id": tile_id, "tif": str(tif_local)}

# Carrega a tabela de URLs
tiles_df = pd.read_csv(TILES_CSV)
tiles_df = tiles_df[tiles_df["tile_id"].isin(TILES_WANTED)].copy()
tiles_df = tiles_df[["tile_id","F10m_download"]].dropna().drop_duplicates()

print("Tiles na lista:", len(tiles_df))
#tiles_df.head(10)

results = []
for _, row in tiles_df.iterrows():
    tile = row["tile_id"]
    url  = row["F10m_download"]
    print(f"=== {tile} ===")
    out = process_tile(tile, url)
    if out:
        results.append(out)

res_df = pd.DataFrame(results)
res_csv = OUT_DIR / "batch_results.csv"
res_df.to_csv(res_csv, index=False, encoding="utf-8")

Tiles na lista: 4
=== 10S_050W ===
[10S_050W] Já existe: 10S_050W_F10m.tif
=== 10S_060W ===
[10S_060W] Já existe: 10S_060W_F10m.tif
=== 20S_050W ===
[20S_050W] Baixando...
=== 20S_060W ===
[20S_060W] Já existe: 20S_060W_F10m.tif
